Plot ROC curves

Maintain by Xiaotong.Chen

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.optim.lr_scheduler import _LRScheduler
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import  utils

from sklearn import decomposition
from sklearn import manifold
from sklearn import metrics
from sklearn import model_selection
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
%matplotlib inline


import copy
from collections import namedtuple
import os, re, time
import random
import shutil
import math

import pandas as pd
import numpy as np
from sklearn import svm

# set seed to make sure the results are reproducible
SEED = 99
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# change directory of data
CV_ID = 2
CV_ID = str(CV_ID)
datapath = os.path.join("D:\\0jiazhuangxian2020\\jiazhuangxian",".data","hcb","cv" + CV_ID)
load_name = 'basic-model-cv'+CV_ID+'.pt'
train_dir = os.path.join(datapath, 'train')
val_dir = os.path.join(datapath, 'val')
test_dir = os.path.join(datapath, 'test')

pretrained_size = (224, 224)
pretrained_means = [0.485, 0.456, 0.406]
pretrained_stds= [0.229, 0.224, 0.225]
test_transforms = transforms.Compose([
                           transforms.Resize(pretrained_size),
                           transforms.ToTensor(),
                           transforms.Normalize(mean = pretrained_means, 
                                                std = pretrained_stds)
                       ])
# inorder to load img with it's label
class MyImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        path, _ = self.imgs[index] #img path, label
        return super(MyImageFolder, self).__getitem__(index), path # return image path
    

BATCH_SIZE = 1000 #4
train_data = MyImageFolder(root = train_dir,
                                transform = test_transforms)
test_data = MyImageFolder(root = test_dir,
                                transform = test_transforms)
valid_data = MyImageFolder(root = val_dir,
                                transform = test_transforms)
train_iterator = data.DataLoader(train_data, 
                                shuffle = True,
                                drop_last = False,
                                batch_size = BATCH_SIZE) 
valid_iterator = data.DataLoader(valid_data, drop_last = False,
                                batch_size = BATCH_SIZE)
test_iterator = data.DataLoader(test_data, drop_last = False,
                                batch_size = BATCH_SIZE)     


# read csv data
path_to_text = '.data\\text.csv'
text_table = pd.read_csv(path_to_text, header=0, index_col=0)
Dimension_Text = len(text_table.columns)

def imgid2index(id):
    return 1000*int(id[0])+int(id[2:])

def imgid2textinfo(imgid):
    # convert img path into text info in .csv
    # input: ['.data\\train\\zy\\1_152.bmp','2_8.bmp']
    # output: tensor([[ 1,  4,  7, 10],
    #    [ 3,  6,  9, 12]])
    return torch.tensor(text_table.loc[[imgid2index(re.search('\d_\d+',i.split('\\')[-1]).group()) for i in imgid],:].values, dtype=torch.float32, device=device)

basic_model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50')
# change output dimension to what we need
IN_FEATURES = basic_model.fc.in_features 
OUTPUT_DIM = 2
basic_model.fc = nn.Linear(IN_FEATURES, OUTPUT_DIM)
fc1=nn.Linear(IN_FEATURES, 32)
fc2=nn.Linear(32,OUTPUT_DIM)
basic_model.fc = nn.Sequential(fc1, fc2)
basic_model.load_state_dict(torch.load(load_name))
basic_model.fc[1] = nn.Identity()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# text_table.head()

In [ ]:
# load train data
num_train = len(train_data)
train_data_combine_model = data.DataLoader(train_data, 
                                shuffle = True, 
                                batch_size = num_train) 
xy, train_id = next(iter(train_data_combine_model))
x, y = xy
img_output = basic_model(x)
train_x = torch.cat((img_output, imgid2textinfo(train_id).cpu()), dim=-1).detach().numpy()
train_y = y
# load val data and combine with train
num_val = len(valid_data)
val_data_combine_model = data.DataLoader(valid_data, 
                                shuffle = True, 
                                batch_size = num_val) 
xy, val_id = next(iter(val_data_combine_model))
x, y = xy
img_output = basic_model(x)
val_x = torch.cat((img_output, imgid2textinfo(val_id).cpu()), dim=-1).detach().numpy()
val_y = y

train_x = np.concatenate((train_x, val_x), axis=0)
train_y = np.concatenate((train_y, val_y), axis=0)
# load test data
num_test = len(test_data)
test_data_combine_model = data.DataLoader(test_data, 
                                batch_size = num_test) 
xy, test_id = next(iter(test_data_combine_model))
x, y = xy
img_output = basic_model(x)
test_x = torch.cat((img_output, imgid2textinfo(test_id).cpu()), dim=-1).detach().numpy()
test_y = y.numpy()

In [ ]:
tmpt=[imgid2index(re.search('\d_\d+',i.split('\\')[-1]).group()) for i in train_id]
tmpv=[imgid2index(re.search('\d_\d+',i.split('\\')[-1]).group()) for i in val_id]
train_ID=tmpt+tmpv
test_ID=[imgid2index(re.search('\d_\d+',i.split('\\')[-1]).group()) for i in test_id]

x_train=text_table.loc[train_ID,:].values
y_train=[2-i//1000 for i in train_ID]
# y_train=[i//1000-1 for i in train_ID]
# print(x_train,y_train)

x_test=text_table.loc[test_ID,:].values
y_test=[2-i//1000 for i in test_ID]
# y_test=[i//1000-1 for i in test_ID]
# print(x_test,y_test)

all_sen={}
all_spe={}

# SVM TEXT+IMG

In [ ]:
#search best params for SVM
svm_para = svm.SVC()    
param_grid = {'C': range(5,30), 'gamma': [1e-2, 7e-3, 5e-3, 3e-3, 1e-3, 7e-4, 5e-4, 3e-4, 1e-4, 7e-5], 
              'kernel': ['rbf', 'linear', 'poly', 'sigmoid']}    
grid_search = model_selection.GridSearchCV(svm_para, param_grid)    
grid_search.fit(train_x, train_y)    
best_parameters = grid_search.best_estimator_.get_params()
for para, val in list(best_parameters.items()):    
    print(para, val) 

In [ ]:
def roc_results(y_true, y_pred, path):
    r"""
    Return: sensitivity, specificity, ppv, npv, accuracy, AUC, threshold
    this function is used to calculate AUC,sensitivity,specificity,ppv,npv,accuracy
    baesed on each threshold and plot the ROC curve. It will also choose the best threshold, which means that this threshold lead to the highest accuracy.
    input: true, pred, path
    """
# calculate the target vector from roc
    fpr, sen, threshold = metrics.roc_curve(y_true, y_pred)
    spe = 1-fpr
    pos_num = np.sum(y_true == 1)
    neg_num = len(y_true)-pos_num
    tp = sen*pos_num
    tn = spe*neg_num
    fn = pos_num-tp
    fp = neg_num-tn
    ppv = tp/(tp+fp+1e-16)
    npv = tn/(tn+fn+1e-16)
    acc = (tp+tn)/len(y_true)
# the best point data
    j_statistic=sen+spe-1#(based on J_statistic)
    ind=np.argmax(j_statistic)
    # ind = np.max(np.where(acc == np.max(acc)))#(based on accuracy)
    pred_p = tp[ind]+fp[ind]
    pred_n = tn[ind]+fn[ind]
    best_acc = acc[ind]
    best_sen = sen[ind]
    best_spe = spe[ind]
    best_ppv = tp[ind]/pred_p
    best_npv = tn[ind]/pred_n
    AUC = metrics.auc(fpr, sen)
# plot
    plt.plot(fpr, sen)
    plt.plot(np.linspace(0, 1, 10), np.linspace(0, 1, 10))
    plt.plot(fpr[ind], sen[ind], 'r.', markersize=9)
    text = "AUC={:.2f}\nSEN={:.2f}\nSPE={:.2f}\nPPV={:.2f}\nNPV={:.2f}\nACC={:.2f}".format(
        AUC, sen[ind], spe[ind], best_ppv, best_npv, best_acc)
    plt.text(0.82, 0.1, text, fontsize=10,
             style="italic", horizontalalignment="center")
    plt.xlabel('1-Specificity')
    plt.ylabel('Sensitivity')
    plt.title('ROC Curve')
    path = path+"/"+"cv"+str(CV_ID)+".jpg"
    plt.savefig(path)
    plt.show()
    return sen, spe, ppv, npv, acc, AUC, threshold

In [ ]:
# combine text info and output of img model with Support Vector Machine
rbf_svc = svm.SVC(C=best_parameters['C'], gamma=best_parameters['gamma'], kernel=best_parameters['kernel'])
# rbf_svc = svm.SVC(kernel='rbf',C = 18, gamma = 0.001)
rbf_svc.fit(train_x, train_y)
path='D:/0jiazhuangxian2020/jiazhuangxian/fig/svmcombine/train'
svm_pred = rbf_svc.predict(train_x)
svm_pred_prob=rbf_svc.decision_function(train_x)
sen, spe, ppv, npv, acc, auc, _=roc_results(1-train_y,1-svm_pred_prob,path)


In [ ]:
svm_pred = rbf_svc.predict(test_x)
path='D:/0jiazhuangxian2020/jiazhuangxian/fig/svmcombine/test'
sen, spe, ppv, npv, acc, auc, _=roc_results(1-test_y,1-rbf_svc.decision_function(test_x),path)
all_sen['svm_c']=sen
all_spe['svm_c']=spe

# SVM TEXT

In [ ]:
#search best params for SVM
svm_para = svm.SVC()    
param_grid = {'C': range(5,8), 'gamma': [1e-2, 1e-1,1e-3], 
              'kernel': ['rbf', 'sigmoid']}    
grid_search = model_selection.GridSearchCV(svm_para, param_grid)    
grid_search.fit(x_train, y_train)    
best_parameters = grid_search.best_estimator_.get_params()
for para, val in list(best_parameters.items()):    
    print(para, val) 

In [ ]:
# combine text info and output of img model with Support Vector Machine
rbf_svc = svm.SVC(C=best_parameters['C'], gamma=best_parameters['gamma'], kernel=best_parameters['kernel'])
# rbf_svc = svm.SVC(kernel='sigmoid',C = 5, gamma = 0.1)
rbf_svc.fit(x_train, y_train)

path='D:/0jiazhuangxian2020/jiazhuangxian/fig/svmtext/train'
svm_pred = rbf_svc.predict(x_train)
y_train=np.array(y_train)

sen, spe, ppv, npv, acc, auc, _=roc_results(1-np.array(y_train),1-np.array(rbf_svc.decision_function(x_train)),path)


In [ ]:
svm_pred = rbf_svc.predict(x_test)
path='D:/0jiazhuangxian2020/jiazhuangxian/fig/svmtext/test'
sen, spe, ppv, npv, acc, auc, _=roc_results(1-np.array(y_test),1-np.array(rbf_svc.decision_function(x_test)),path)
all_sen['svm_t']=sen
all_spe['svm_t']=spe

# basic model(only image)

In [ ]:
def get_predictions(model, iterator):
    model.eval()
    images = []
    labels = []
    probs = []
    top_pred = []
    with torch.no_grad():
        for (xy, _) in iterator:
            x, y = xy
            y_pred = model(x)
            y_prob = F.softmax(y_pred, dim=-1)
            _, _top_pred = torch.max(y_pred, 1)
            images.append(x.cpu())
            labels.append(y.cpu())
            probs.append(y_prob.cpu())
            top_pred.append(_top_pred.cpu())
    images = torch.cat(images, dim=0)
    labels = torch.cat(labels, dim=0)
    probs = torch.cat(probs, dim=0)
    top_pred = torch.cat(top_pred, dim=0)
    return images, labels, probs, top_pred

In [ ]:
basic_model.to("cpu")

_, labels, probs, _ =get_predictions(basic_model, test_iterator)

In [ ]:
pred=probs[:,1]
# sen, spe, ppv, npv, acc, auc, _ = roc_results(1-labels.numpy(),pred.numpy(),path)
sen, spe, ppv, npv, acc, auc, _ = roc_results(labels.numpy(),pred.numpy(),path)
all_sen['img']=sen
all_spe['img']=spe

# all roc

In [ ]:
plt.style.use('ggplot')
plt.plot(1-all_spe['svm_c'],all_sen['svm_c'],linewidth=1.5)
plt.plot(1-all_spe['svm_t'],all_sen['svm_t'],linewidth=1.5)
plt.plot(1-all_spe['img'],all_sen['img'],color='goldenrod',linewidth=1.5)
plt.plot(np.linspace(0, 1, 10), np.linspace(0, 1, 10),'--',label=None,color='forestgreen',linewidth=1.5)
plt.xlim([-0.02, 1.0])
plt.ylim([0.0, 1.02])
plt.xlabel('1-Specificity')
plt.ylabel('Sensitivity')
plt.title('ROC Curves')
plt.legend((r"$Ours$",r"$SVM_C$",r"$DNN_I$"),loc='lower right')
# plt.axis('equal')
plt.gca().set_aspect('equal', adjustable='box')

plt.savefig("three_curve_roc11.svg")
plt.show()